# Preprocessing, building a Pandas dataframe and saving it as a  .csv file

In [22]:
import re
import sys
import glob
import string
from pprint import pprint
from collections import Counter, OrderedDict

import spacy
nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')
from sklearn.metrics import accuracy_score


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

folder_path = "/home/niki/Documents/BE_Project/gcpEmailRecommendation/Scraping/mini_deb/*"
file_name = "/home/niki/Documents/BE_Project/my_EmailRecommmendation/model/dataframe3.csv"
sys.path.insert(0, '/home/niki/Documents/BE_Project/my_EmailRecommmendation/Preprocessing')


import preprocessing
import read_file
import datetime

def extract_debian(text):
    text = text.split('\n\n\n')
    header = text[0].split('\n')
    body = text[1]
    sender = header[2].split(':')[1].split('<')[0]
#     print('Sender',sender)
#     print('Body \n',body)
    return sender,body

def clean_debian(temp):
    temp = re.sub('\n+','\n',temp)
    temp = re.sub('\n',' ',temp)
    temp = re.sub('\t',' ',temp)
    temp = re.sub(' +',' ',temp)
    return temp

def deb_lemmatize(doc):        
    doc = nlp(doc)
    article, skl_texts = '',''
    for w in doc:
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            article += " " + w.lemma_
        if w.text == '\n':                
            skl_texts += " " + article
            article = ''       
    return skl_texts

def deb_toppostremoval(temp):
    strings = temp.splitlines()
    temp = ''
    for st in strings:
        st = st.strip()
        if len(st)>0:
            if st[0]=='>':
                continue
            else:
                temp += '\n' + st
    return temp

df = pd.DataFrame(columns=['body','replier', 'thread_no'])
folder = glob.glob(folder_path)
th_no = 0
obj = preprocessing.preprocess()
cnt = 0
count_file = 0
thread_list=[]
try:
    for fol in tqdm_notebook(folder):
        files = glob.glob(fol+'/*.txt')
        threads = []
        for file in files:
            ob = read_file.file_content(file)
            ob.read_file_content()
            threads.append(ob.mail)
            count_file += 1
        sorted_threads = sorted(threads, key=lambda ke: datetime.datetime.strptime(ke['Date'],'%a, %d %b %Y %H:%M:%S %z'))
        thread_list.append(sorted_threads)
except:
    print(fol)
print(len(thread_list))

HBox(children=(IntProgress(value=0, max=23), HTML(value='')))


23


In [23]:
empty = open('null.txt','w')
for thr in thread_list:
    flag = 0
    t = ''
    for mail in thr:
        temp = ''
        count_file += 1
        sender = mail['From']
        temp   = mail['content']
        temp = deb_toppostremoval(temp)
        temp = deb_lemmatize(temp)
        temp = clean_debian(temp)
        if temp == '':
            cnt += 1
            empty.write(file + '\n')
            print('NULL')
            continue
        temp = obj.replace_tokens(temp)
        if flag==0:
            t = temp
            flag = 1
            continue
    # Add start/end date of each thread  |> FLAG
        df = df.append({'body': str(t),'replier':sender, 'thread_no':th_no}, ignore_index=True)
        t = t + temp
    th_no += 1

empty.close()
print(cnt)
print(count_file)
print(len(df['body']))
print(len(df['thread_no'].unique()))
print(len(df['replier'].unique()))
rep_to_index = {}
index = 0
for rep in df['replier']:
    if rep in rep_to_index:
        continue
    else:
        rep_to_index[rep] = index
        index += 1
pprint(len(rep_to_index))
for rep in df['replier']:
    df.loc[df['replier']==rep,'replier'] = rep_to_index[rep]

# Aggregate according to date / make separate thread list |> P flag
# Split test_train_split 
print(df.head)
df.to_csv(file_name)
unique_users = len(df.replier.unique())

83

NULL


83

NULL


83

...

NULL


83

NULL


83

NULL
6
184
63
23
37
37
<bound method NDFrame.head of                                                  body replier thread_no
0    Dear[day]end[person] need advice development ...       0         0
1    Dear[day]end[person] need advice development ...       1         0
2    Dear[day]end[person] need advice development ...       0         0
3    Hi -PRON- would like try LZ[number] compress ...       2         1
4    Hi -PRON- would like try LZ[number] compress ...       2         1
5    Hi -PRON- would like try LZ[number] compress ...       2         1
6    Package wnpp Owner Hilko Bengen < bengen@debi...       3         2
7    Package wnpp Owner Hilko Bengen < bengen@debi...       3         2
8    Package wnpp Owner Hilko Bengen < bengen@debi...       3         2
9    On Tue [number] -[number] Afif Elghraoui writ...       4         3
10   On Tue [number] -[number] Afif Elghraoui writ...       5         3
11   On Tue [number] -[number] Afif Elghraoui writ...       4         3
12   jvieir

# Indexing of words in vocab

In [24]:
words = Counter()
for sent in df.body.values:
    words.update(w.text.lower() for w in nlp(sent))
# print(words)

words = sorted(words, key=words.get, reverse=True)
print(words)
words = ['_PAD','_UNK'] + words

word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}
def indexer(s): return [word2idx[w.text.lower()] for w in nlp(s)]

[']', '[', '=', 'package', 'database', 'url', 'debian', 'not', 'write', 'on', '>', ' ', '<', '-pron-', 'have', 'release', 'bug', 'file', 'fix', 'number', 'need', 'the', 'upgrade', 'install', 'application', '|', 'list', 'be', 'maintainer', 'work', 'upstream', 'provide', 'version', 'this', 'for', '+', 'automate', 'want', 'license', 'run', 'discussion', '⠀', 'comic', 'available', 'system', 'would', 'like', 'issue', '`', 'key', 'dh', 'common', 'datum', 'tool', 'hi', 'font', 'start', 'time', '.buildinfo', 'look', 'practice', 'failure', 'build', 'severity', '-[number', 'author', 'description', 'sudo', 'filetype]ument', 'know', 'try', 'apt', 'developer', 'policy', 'what', 'upload', 'include', 'curl', 'test', 'library', 'make', 'bash', 'process', 'remove', 'consider', 'project', 'we', 'think', 'buster', 'case', 'sans', 'maintain', 'point', 'security', 'gnome', 'there', 'reply', 'number]pm', 'wnpp', 'owner', 'wishlist', 'programming', 'lang', 'tue', 'download', 'dbconfig', 'persistent', 'cache'

# Dataset Loading

In [25]:
# embedding |> flag
class VectorizeData(Dataset):
    def __init__(self, df_path, maxlen=10):
        self.df = pd.read_csv(df_path, error_bad_lines=False)
        self.df['body'] = self.df.body.apply(lambda x: x.strip())
        print('Indexing...')
        self.df['bodyidx'] = self.df.body.apply(indexer)
        print('Calculating lengths')
        self.df['lengths'] = self.df.bodyidx.apply(len)
        self.maxlen = max(self.df['lengths'])
        print(self.maxlen)
        print('Padding')
        self.df['bodypadded'] = self.df.bodyidx.apply(self.pad_data)
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        X = self.df.bodypadded[idx]
        lens = self.df.lengths[idx]
        y = self.df.replier[idx]
        return X,y,lens
    
    def pad_data(self, s):
        padded = np.zeros((self.maxlen,), dtype=np.int64)
        if len(s) > self.maxlen: padded[:] = s[:self.maxlen]
        else: padded[:len(s)] = s
        return padded

In [26]:
ds = VectorizeData(file_name)

Indexing...
Calculating lengths
710
Padding


In [27]:
input_size = ds.maxlen
hidden_size = 30
num_classes = unique_users
num_epochs = 5
batch_size = 1
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
# concatenate user vector
# embeddings |>
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
#         x = torch.FloatTensor(x)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [32]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [33]:
train_dl= DataLoader(ds, batch_size=1)
num_batch = len(train_dl)
for epoch in range(num_epochs):
    y_true_train = list()
    y_pred_train = list()
    total_loss_train = 0
    t = tqdm_notebook(iter(train_dl), leave=False, total=num_batch)
    for X,y, lengths in t:
    #     X = X.transpose(0,1)
        X = Variable(X.cpu())
        y = Variable(y.cpu())
        lengths = lengths.numpy()

        opt.zero_grad()
        X = X.float()
        pred = model(X)
        # F.nll_loss can be replaced with criterion
        loss = F.nll_loss(pred, y)
        loss.backward()
        opt.step()

        t.set_postfix(loss=loss.data[0])
        pred_idx = torch.max(pred, dim=1)[1]

        y_true_train += list(y.cpu().data.numpy())
        y_pred_train += list(pred_idx.cpu().data.numpy())
        total_loss_train += loss.data[0]

    train_acc = accuracy_score(y_true_train, y_pred_train)
    train_loss = total_loss_train/len(train_dl)
    print(f' Epoch {epoch}: Train loss: {train_loss} acc: {train_acc}')
#Accuracy
# hyperparameter tuning
#Testing
# architecture testing

HBox(children=(IntProgress(value=0, max=63), HTML(value='')))

 Epoch 0: Train loss: -32.8299446105957 acc: 0.031746031746031744


HBox(children=(IntProgress(value=0, max=63), HTML(value='')))

 Epoch 1: Train loss: -367.3641357421875 acc: 0.07936507936507936


HBox(children=(IntProgress(value=0, max=63), HTML(value='')))

 Epoch 2: Train loss: -937.3109130859375 acc: 0.12698412698412698


HBox(children=(IntProgress(value=0, max=63), HTML(value='')))

 Epoch 3: Train loss: -1878.892822265625 acc: 0.14285714285714285


HBox(children=(IntProgress(value=0, max=63), HTML(value='')))

 Epoch 4: Train loss: -3369.987060546875 acc: 0.1111111111111111
